In [1]:
#importing the libraries

import pandas as pd
import yfinance as yf
import numpy as np

In [2]:
port_pick = pd.read_csv('Data/portfolio_pick.csv')['Stock'].tolist()

In [3]:
def fetch_high_volatility_tickers(ticker_list, threshold=0.1):
    high_vol_tickers = []
    vol_data = {}

    for ticker in ticker_list:
        try:
            stock_data = yf.download(ticker, period="1y", interval="1d", progress=False)
            
            if stock_data.empty:
                print(f"No data found for {ticker}.")
                continue

            closing_prices = stock_data['Close']
            std_dev = np.std(closing_prices)
            mean_price = np.mean(closing_prices)

            vol_data[ticker] = {
                "std_dev": std_dev,
                "mean_price": mean_price,
                "std_dev_pct": std_dev / mean_price if mean_price > 0 else 0
            }

            if vol_data[ticker]["std_dev_pct"] >= threshold:
                high_vol_tickers.append(ticker)

            print(f"Processed {ticker}: Std Dev = {std_dev:.2f}, Mean Price = {mean_price:.2f}, Std Dev % = {std_dev / mean_price:.2%}")

        except Exception as e:
            print(f"Error processing {ticker}: {e}")

    return high_vol_tickers, vol_data


vol_tickers = port_pick
threshold = 0.1

high_vol_tickers, vol_data = fetch_high_volatility_tickers(port_pick, threshold)

print("\nTickers with standard deviation >= 10%:")
for ticker in high_vol_tickers:
    print(f"{ticker}: Std Dev % = {vol_data[ticker]['std_dev_pct']:.2%}")

Processed EBAY: Std Dev = 7.42, Mean Price = 54.32, Std Dev % = 13.67%
Processed CMG: Std Dev = 5.55, Mean Price = 56.91, Std Dev % = 9.75%
Processed EXPE: Std Dev = 20.07, Mean Price = 141.82, Std Dev % = 14.15%
Processed F: Std Dev = 1.02, Mean Price = 11.70, Std Dev % = 8.75%
Processed GM: Std Dev = 5.55, Mean Price = 45.32, Std Dev % = 12.24%
Processed HD: Std Dev = 27.87, Mean Price = 369.05, Std Dev % = 7.55%
Processed MAR: Std Dev = 18.78, Mean Price = 246.08, Std Dev % = 7.63%
Processed PG: Std Dev = 7.46, Mean Price = 164.66, Std Dev % = 4.53%
Processed DG: Std Dev = 27.64, Mean Price = 118.56, Std Dev % = 23.32%
Processed XOM: Std Dev = 6.80, Mean Price = 113.35, Std Dev % = 6.00%
Processed AXP: Std Dev = 31.49, Mean Price = 241.59, Std Dev % = 13.03%
Processed BAC: Std Dev = 3.90, Mean Price = 38.99, Std Dev % = 10.00%
Processed BLK: Std Dev = 92.97, Mean Price = 861.54, Std Dev % = 10.79%
Processed FIS: Std Dev = 8.64, Mean Price = 75.68, Std Dev % = 11.42%
Processed GS: St

In [4]:
def fetch_options_data(ticker_list):
    options_data = {}

    for ticker in ticker_list:
        try:
            stock = yf.Ticker(ticker)
            
            expiration_dates = stock.options
            
            ticker_data = {"calls": [], "puts": []}
            
            for date in expiration_dates:
                opt_chain = stock.option_chain(date)
                
                ticker_data["calls"].append({"expiration_date": date, "data": opt_chain.calls})
                ticker_data["puts"].append({"expiration_date": date, "data": opt_chain.puts})
            
            options_data[ticker] = ticker_data
            print(f"Fetched options data for {ticker}")
        
        except Exception as e:
            print(f"Error fetching data for {ticker}: {e}")
    
    return options_data



options_list = high_vol_tickers

all_options_data = fetch_options_data(options_list)

Fetched options data for EBAY
Fetched options data for EXPE
Fetched options data for GM
Fetched options data for DG
Fetched options data for AXP
Fetched options data for BAC
Fetched options data for BLK
Fetched options data for FIS
Fetched options data for GS
Fetched options data for MS
Fetched options data for WFC
Fetched options data for BIIB
Fetched options data for LMT
Fetched options data for AMD
Fetched options data for CRM
Fetched options data for MU
Fetched options data for QCOM


In [7]:
def analyze_strangle_profit(ticker, std_dev, current_price):
    try:
        stock = yf.Ticker(ticker)
        expiration_dates = stock.options
        
        results = []

        for date in expiration_dates[:1]:  # only look at the first expiration date
            opt_chain = stock.option_chain(date)
            calls = opt_chain.calls
            puts = opt_chain.puts
            
            # for strangle, we should:
            # 1. buy call option with strike price higher than the current price
            # 2. buy put option with strike price lower than the current price
            otm_calls = calls[calls['strike'] > current_price]  # out of the money call
            otm_puts = puts[puts['strike'] < current_price]     # out of the money put
            
            for _, call in otm_calls.iterrows():
                for _, put in otm_puts.iterrows():
                    strike_price_call = call['strike']
                    strike_price_put = put['strike']
                    call_price = call['lastPrice']
                    put_price = put['lastPrice']
                    strangle_cost = call_price + put_price
                    
                    # upward profit = stock price increase above call strike price - strangle cost
                    upward_profit = max(0, (current_price + std_dev - strike_price_call) - strangle_cost)
                    
                    # downward profit = stock price decrease below put strike price - strangle cost
                    downward_profit = max(0, (strike_price_put - (current_price - std_dev)) - strangle_cost)
                    
                    if (upward_profit > 0) and (downward_profit > 0):
                        results.append({
                            "call_strike": strike_price_call,
                            "put_strike": strike_price_put, 
                            "call_price": call_price,
                            "put_price": put_price,
                            "strangle_cost": strangle_cost,
                            "upward_profit": upward_profit,
                            "downward_profit": downward_profit,
                            "meets_criteria": True
                        })
        
        return results

    except Exception as e:
        print(f"Error analyzing {ticker}: {e}")
        return []

def find_profitable_strangles(ticker_list, threshold=0.1):
    profitable_strangles = {}
    
    for ticker in ticker_list:
        try:
            stock_data = yf.download(ticker, period="1y", interval="1d", progress=False)
            closing_prices = stock_data['Close']
            std_dev = np.std(closing_prices)
            current_price = closing_prices.iloc[-1]

            if std_dev / current_price >= threshold:
                results = analyze_strangle_profit(ticker, std_dev, current_price)
                if results:
                    profitable_strangles[ticker] = results

        except Exception as e:
            print(f"Error processing {ticker}: {e}")

    return profitable_strangles


port_pick = high_vol_tickers
threshold = 0.1
profitable_strangles = find_profitable_strangles(port_pick, threshold)

for ticker, strangles in profitable_strangles.items():
    print(f"\nProfitable Strangles for {ticker}:")
    for s in strangles:
        print(s)


Profitable Strangles for EBAY:
{'call_strike': 66.0, 'put_strike': 59.0, 'call_price': 0.36, 'put_price': 0.05, 'strangle_cost': 0.41, 'upward_profit': 6.0241171061082746, 'downward_profit': 1.0041128336473373, 'meets_criteria': True}
{'call_strike': 66.0, 'put_strike': 60.0, 'call_price': 0.36, 'put_price': 0.12, 'strangle_cost': 0.48, 'upward_profit': 5.954117106108274, 'downward_profit': 1.9341128336473372, 'meets_criteria': True}
{'call_strike': 66.0, 'put_strike': 61.0, 'call_price': 0.36, 'put_price': 0.05, 'strangle_cost': 0.41, 'upward_profit': 6.0241171061082746, 'downward_profit': 3.004112833647337, 'meets_criteria': True}
{'call_strike': 66.0, 'put_strike': 62.0, 'call_price': 0.36, 'put_price': 0.12, 'strangle_cost': 0.48, 'upward_profit': 5.954117106108274, 'downward_profit': 3.934112833647337, 'meets_criteria': True}
{'call_strike': 66.0, 'put_strike': 63.0, 'call_price': 0.36, 'put_price': 0.27, 'strangle_cost': 0.63, 'upward_profit': 5.804117106108275, 'downward_profit

In [8]:
# find the most profitable strangle for each ticker
most_profitable_strangles = {}

for ticker, strangles in profitable_strangles.items():
    # sort the strangles by the maximum of upward or downward profit
    max_profit_strangle = max(strangles, 
                            key=lambda x: max(x['upward_profit'], x['downward_profit']))
    most_profitable_strangles[ticker] = max_profit_strangle

pd.DataFrame.from_dict(most_profitable_strangles, orient='index')


,call_strike,put_strike,call_price,put_price,strangle_cost,upward_profit,downward_profit,meets_criteria
EBAY,70.0,65.0,0.03,0.74,0.77,1.664117,6.644113,True
EXPE,185.0,175.0,2.46,0.18,2.64,17.184569,7.684569,True
GM,52.0,47.0,0.65,0.04,0.69,4.666129,0.046126,True
DG,90.0,76.0,0.01,1.05,1.06,12.983295,26.183292,True
AXP,325.0,297.5,0.03,2.82,2.85,2.285959,27.485972,True
FIS,82.0,77.0,0.53,0.15,0.68,7.420286,3.500288,True
GS,625.0,565.0,0.05,5.75,5.80,1.500847,59.300896,True
MS,133.0,123.0,0.02,1.30,1.32,2.807058,11.927053,True
BIIB,147.0,140.0,1.90,0.32,2.22,24.685538,18.745536,True
LMT,490.0,450.0,3.00,0.05,3.05,51.878071,13.838093,True
